In [2]:
import sys
sys.path.append('/data0/tianjunchao/code/Tian-EEG-Image/')
import torch
from prepare.eegdataset import GeneralEEGImageDataset

from torch.utils.data import DataLoader


from utils.eegutils import get_test_setting
from matplotlib import pyplot as plt
import numpy as np
from tsne_torch import TorchTSNE as TSNE

import run.resnet as resnet
import torch.nn as nn
data_path = '/data0/tianjunchao/dataset/CVPR2021-02785/data/img_pkl/32x32'

In [13]:
# dataset = get_test_setting(dataset=GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8))

from utils.eegutils import get_device
from tsnecuda import TSNE

dataset = GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8)
# dataset = get_test_setting(dataset)
train_loader = DataLoader(dataset, batch_size=128)

In [14]:

# pca
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca_results = pca.fit_transform(dataset[:1000].data)
plt.scatter(pca_results[:,0], pca_results[:,1], c=dataset[:1000].targets)
plt.show()

AttributeError: 'GeneralEEGImageDataset' object has no attribute 'data'

In [9]:
device = get_device(0)
def tsne_visualization(model_name, trainloader, perplexity=30, n_iter=1000):
    
    model = resnet.ResNet(num_classes=40,model_name=model_name, pretrained=True)
    model = list(model.children())[0]

    feature_extractor = nn.Sequential(*list(model.children())[:-1])
    feature_extractor = feature_extractor.to(device)
    feature_extractor.eval()      
    # 提取训练数据的特征向量
    features = []
    labels = []
    # tsne = TSNE(n_components=2)
    tsne = TSNE(n_components=2, perplexity=perplexity, n_iter=n_iter,verbose=True)
    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        # to device
        inputs = inputs.to(device)
        labels.extend(targets)

        with torch.no_grad():
            # features.append(feature_extractor(inputs).squeeze())
            tsne_results = tsne.fit_transform(inputs[:n_samples])
        break
    # features = np.concatenate(features)
    # labels = np.concatenate(labels)

    # 使用T-SNE算法将特征向量降至2维
    # features = torch.cat(features, dim=0)
    return tsne_results, labels[:n_samples]
# list names of resnet

model_list = [resnet.resnet_name_A,resnet.resnet_name_B,resnet.resnet_name_C,resnet.resnet_name_D,resnet.resnet_name_E]
tsne_list = []
label_list = []
file_name = "4.materials/tsne_results_"
for model_name in model_list:
    tsne_results, labels = tsne_visualization(model_name, train_loader, perplexity=30, n_iter=1000)
    torch.save((tsne_results, labels), file_name+model_name+".txt")
    print("save file: ", file_name+model_name+".txt")

using cuda
initializing...


RuntimeError: This function was deprecated since version 1.9 and is now removed. Please use the `torch.linalg.eig` function instead.

In [ ]:
# draw 5 tsne in one figure
plt.figure(figsize=(20, 20))
for i in range(5):
    plt.subplot(5, 1, i+1)
    plt.scatter(tsne_list[i][:, 0], tsne_list[i][:, 1], c=label_list[i], cmap='tab10')
plt.show()

In [ ]:
print(len(train_loader),len(dataset))

In [ ]:
modules = list(model.children())
print(len(modules))
print(modules)

In [ ]:
print(torch.max(imgs),torch.min(imgs))    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 将通道维度放在最后，以符合imshow的要求
imgs = np.transpose(imgs, (0, 2, 3, 1))

# 打印16张图像
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))

for i, ax in enumerate(axes.flat):
    ax.imshow(imgs[i])
    ax.set_axis_off()

# 调整子图之间的间距
plt.tight_layout()

# 显示图像
plt.show()

In [ ]:
img = imgs[1]
plt.imshow(img)

# 显示图像
plt.show()
# 分别画出每个通道的图像
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(9, 3))

for i, ax in enumerate(axes):
    ax.imshow(img[..., i])
    ax.set_title('Channel {}'.format(i+1))

# 调整子图之间的间距
plt.tight_layout()

# 显示图像
plt.show()

In [ ]:

# resnet 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

resnet = models.resnet101(pretrained=True)
# print(resnet)
resnet.fc = nn.Conv2d(resnet.fc.in_features, 40, kernel_size=1)
# print(resnet)
modules = list(resnet.children())
print(len(modules))
print(modules)
# [:-1]
# resnet.flatten_features_output = False
# feature_extractor = nn.Sequential(*modules)
# random_input = torch.randn(19, 3, 224, 224)
# output = feature_extractor(random_input)
# print(output.shape)
# print(feature_extractor)
# classifiers = resnet.fc
# output = classifiers(output)
# print(output.shape)
# print(resnet(random_input))
# print(classifiers)
# # print(feature_extractor)

In [ ]:
xx = nn.Sequential(*list(resnet.children())[:-2])
print(xx(random_input).shape)


In [ ]:
for i in range(1,1+len(list(resnet.children()))):
    print(list(resnet.children())[-i])
    print('------------------')

In [ ]:
# resnet50
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

resnet = models.resnet50(pretrained=True)
print(resnet)

In [ ]:
0.98**100

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

# Define the range of x values to plot
x = np.linspace(0, 1, 100)

# Define the alpha and beta parameters for the beta distribution
alphas = [1, 2, 3, 4, 5, 6]
betas = [1, 2, 3, 4, 5, 6]

# Create a 2x3 grid of subplots
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8))

# Loop over the alpha and beta values and plot the corresponding beta distribution
for i, (alpha, beta_param) in enumerate(zip(alphas, betas)):
    row = i // 3  # Determine the row index
    col = i % 3   # Determine the column index

    # Generate a beta distribution object
    dist = beta(alpha, beta_param)

    # Compute the PDF values for the given x values
    pdf = dist.pdf(x)

    # Plot the PDF of the beta distribution in the corresponding subplot
    axes[row, col].plot(x, pdf, label=r'$\alpha={}, \beta={}$'.format(alpha, beta_param))
    axes[row, col].set_title(r'Beta distribution with $\alpha={}, \beta={}$'.format(alpha, beta_param))
    axes[row, col].set_xlabel('x')
    axes[row, col].set_ylabel('PDF')
    axes[row, col].legend()

# Adjust the layout of the subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

# Define the range of x values to plot
x = np.linspace(0, 1, 100)

# Define the alpha and beta parameters for the beta distribution
alphas = np.array(range(0,16))*0.1

# Create a 2x3 grid of subplots
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(12, 8))

# Loop over the alpha and beta values and plot the corresponding beta distribution
for i, alpha in enumerate(alphas):
    row = i // 4  # Determine the row index
    col = i % 4   # Determine the column index

    # Generate a beta distribution object
    dist = beta(alpha, alpha)

    # Compute the PDF values for the given x values
    pdf = dist.pdf(x)

    # Plot the PDF of the beta distribution in the corresponding subplot
    axes[row, col].plot(x, pdf, label=r'$\alpha={:.1f}$'.format(alpha))
    axes[row, col].set_title(r'Beta distribution with $\alpha={:.1f}$'.format(alpha))
    axes[row, col].set_xlabel('x')
    axes[row, col].set_ylabel('PDF')
    axes[row, col].legend()

# Adjust the layout of the subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

# Define the range of x values to plot
x = np.linspace(0, 1, 100)

# Define the alpha and beta parameters for the beta distribution
alphas = np.array(range(1,10))*0.01+0.9
betas = np.array(range(1,10))*0.01+0.9

# Create a 2x3 grid of subplots
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 8))

# Loop over the alpha and beta values and plot the corresponding beta distribution
for i, (alpha, beta_param) in enumerate(zip(alphas, betas)):
    row = i // 3  # Determine the row index
    col = i % 3   # Determine the column index

    # Generate a beta distribution object
    dist = beta(alpha, beta_param)

    # Compute the PDF values for the given x values
    pdf = dist.pdf(x)

    # Plot the PDF of the beta distribution in the corresponding subplot
    axes[row, col].plot(x, pdf, label=r'$\alpha={:.2f}, \beta={:.2f}$'.format(alpha, beta_param))
    axes[row, col].set_title(r'Beta distribution with $\alpha={:.2f}, \beta={:.2f}$'.format(alpha, beta_param))
    axes[row, col].set_xlabel('x')
    axes[row, col].set_ylabel('PDF')
    axes[row, col].legend()

# Adjust the layout of the subplots
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:

from models.resnet import ResNet


model = ResNet(num_classes=40,model_name='resnet152', pretrained=True)
# print if parameters are trainable

print('------------------')
for name, param in model.named_parameters():
    print(name, param.requires_grad)

# freeze all parameters

In [ ]:
# 使用scikit-learn中的t-SNE算法进行降维
from sklearn.manifold import TSNE

def visualize_feature(feat_matrix, feat_labels):
    tsne = TSNE(n_components=2, random_state=0)
    feat_matrix_tsne = tsne.fit_transform(feat_matrix)

    # 绘制可视化图像
    plt.scatter(feat_matrix_tsne[:, 0], feat_matrix_tsne[:, 1])
    plt.show()

In [7]:

model = resnet.ResNet(num_classes=40,model_name=resnet.resnet_name_C, pretrained=True)
model = list(model.children())[0]
modules = list(model.children())
for i in range(len(modules)):
    print(modules[i])
    print('------------------')

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
------------------
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
------------------
ReLU(inplace=True)
------------------
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
------------------
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bia